In [3]:
### Step 1: Import of necesssary Lib
import cv2
import numpy as np
import time

### Step 2: Intilization of requirement paramenters
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5
config_path = "cfg/yolov3.cfg" #Initialization with the layes parameter
weights_path = "weights/yolov3.weights" #Intialization with the pretrained weights for more accuracy fast processing
font_scale = 1
thickness = 1
LABELS = open("data/coco.names").read().strip().split("\n") #Initialing with the Lebels to be detected in the Video
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

cap = cv2.VideoCapture('test_video.mp4') #Passing the video as parameter to the Algo

### Step 3: Reading the Video as Images into the buffer and Preprocessing the image
while True:
    _, image = cap.read()

    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    start = time.perf_counter()
    layer_outputs = net.forward(ln)
    time_took = time.perf_counter() - start
    print("Time took:", time_took)
    boxes, confidences, class_ids = [], [], []

### Step 4: Object Detection

    # loop over each of the layer outputs
    for output in layer_outputs:
        # loop over each of the object detections
        for detection in output:
            # extract the class id (label) and confidence (as a probability) of
            # the current object detection
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # discard weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONFIDENCE:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # perform the non maximum suppression given the scores defined before
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)

    font_scale = 1
    thickness = 1
    
### Step 5: Checking for the Object Detected and Marking them as Box

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in COLORS[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
            text = f"{LABELS[class_ids[i]]}: {confidences[i]:.2f}"
            # calculate text width & height to draw the transparent boxes as background of the text
            (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
            text_offset_x = x
            text_offset_y = y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
            overlay = image.copy()
            cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
            # add opacity (transparency to the box)
            image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
            # now put the text (label: confidence %)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

    cv2.imshow("image", image)
    if ord("q") == cv2.waitKey(1):
        break

cap.release()
cv2.destroyAllWindows()

Time took: 1.0959861709998222
Time took: 0.9971387840014359
Time took: 0.8660326989993337
Time took: 0.8620499970002129
Time took: 0.862671777000287
Time took: 0.8635630389999278
Time took: 0.8633673929998622
Time took: 0.8627688519991352
Time took: 0.8686968319998414
Time took: 0.8612541990005411
Time took: 0.8613511710009334
Time took: 0.8853496390001965
Time took: 0.8796605240004283
Time took: 0.8648026129994832
Time took: 0.8647811000009824
Time took: 0.8645071330010978
Time took: 0.8607513839997409
Time took: 0.8598471680015791
Time took: 0.8617814370009
Time took: 0.9180119390002801
Time took: 0.868966571000783
Time took: 1.1123441250001633
Time took: 0.9514849630013487
Time took: 0.8687645429999975
Time took: 0.8662621059993398
Time took: 0.8685963790012465
Time took: 0.8636517289996846
Time took: 0.8667161470002611
Time took: 0.8657464940006321
Time took: 0.8655026939995878
Time took: 0.8639598699992348
Time took: 0.8624510980007472
Time took: 1.0755004450002161
Time took: 0.86

KeyboardInterrupt: 